# DallasCouncilVoters.csv

In [1]:
import pyspark
from pyspark import SparkContext

sc = SparkContext(master='local', appName = 'Spark context')

In [2]:
import findspark
findspark.init()
import pyspark
findspark.find()

'/Users/vunguyen/server/spark-2.4.6-bin-hadoop2.7'

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
'''
conf= pyspark.SparkConf().setAppName('tets_app').setMaster('local')
sc=pyspark.SparkContext(conf=conf)
sc
'''

#sc= SparkContext()
spark = (SparkSession
         .builder
         .appName('Session Spark App')
         .getOrCreate())
spark

1. Đọc file

In [4]:
df = spark.read.csv('File/DallasCouncilVoters.csv', header = True, inferSchema=True)

2. Cho biết data bao nhiêu rows, in Schema

In [5]:
df.count()

44625

In [6]:
df.printSchema()

root
 |-- DATE: string (nullable = true)
 |-- TITLE: string (nullable = true)
 |-- VOTER_NAME: string (nullable = true)



In [7]:
df.show(5)

+----------+-------------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|
+----------+-------------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|
|02/08/2017|Councilmember|       Casey Thomas|
+----------+-------------+-------------------+
only showing top 5 rows



3. Kiểm tra dữ liệu nan, Null. nếu có dòng 'VOTER_NAME' nào có dữ liệu null thì xoá

In [ ]:
from pyspark.sql.functions import col,udf
from pyspark.sql.functions import isnan,when,count,col

In [9]:
df.select([count(when(isnan(c),c)).alias(c) for c in df.columns]).toPandas().T

,0
DATE,0
TITLE,0
VOTER_NAME,0


In [10]:
df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns]).toPandas().T

,0
DATE,0
TITLE,195
VOTER_NAME,503


In [11]:
df = df.dropna(subset = 'VOTER_NAME')

In [12]:
df.select([count(when(col(c).isNull(),c)).alias(c) for c in df.columns]).toPandas().T

,0
DATE,0
TITLE,0
VOTER_NAME,0


4. Kiểm tra dữ liệu trùng, xoá

In [13]:
num_rows = df.count()
num_dist_rows = df.distinct().count()
dup_rows = num_rows - num_dist_rows

In [14]:
display(num_rows, num_dist_rows, dup_rows)

44122

1273

42849

In [15]:
df.filter(df['VOTER_NAME'] == 'Philip T. Kingston').show(5)

+----------+-------------+------------------+
|      DATE|        TITLE|        VOTER_NAME|
+----------+-------------+------------------+
|02/08/2017|Councilmember|Philip T. Kingston|
|02/08/2017|Councilmember|Philip T. Kingston|
|01/11/2017|Councilmember|Philip T. Kingston|
|09/14/2016|Councilmember|Philip T. Kingston|
|01/04/2017|Councilmember|Philip T. Kingston|
+----------+-------------+------------------+
only showing top 5 rows



In [16]:
df = df.drop_duplicates()

In [17]:
df.count()

1273

5. Tìm các VOTER_NAME duy nhất, hiển thị 10

In [18]:
df.select(df['VOTER_NAME']).distinct().show(10)

+--------------------+
|          VOTER_NAME|
+--------------------+
|      Tennell Atkins|
|  the  final   20...|
|        Scott Griggs|
|       Scott  Griggs|
|       Sandy Greyson|
| Michael S. Rawlings|
| the final 2018 A...|
|        Kevin Felder|
|        Adam Medrano|
|       Casey  Thomas|
+--------------------+
only showing top 10 rows



6. Lọc VOTER_NAME có chiều dài 20 ký tự

In [19]:
from pyspark.sql.functions import *

In [20]:
df.filter('length(VOTER_NAME) >0 and length(VOTER_NAME) < 20')

DataFrame[DATE: string, TITLE: string, VOTER_NAME: string]

In [21]:
df.show(5)

+----------+--------------------+------------------+
|      DATE|               TITLE|        VOTER_NAME|
+----------+--------------------+------------------+
|04/11/2018|Deputy Mayor Pro Tem|      Adam Medrano|
|02/14/2018|       Councilmember|   Lee M. Kleinman|
|04/25/2018|       Councilmember|    Tennell Atkins|
|08/29/2018|       Councilmember|      Kevin Felder|
|10/18/2017|       Councilmember|Jennifer S.  Gates|
+----------+--------------------+------------------+
only showing top 5 rows



7. Loại bỏ các VOTER_NAME có chứa dấu '_'

In [22]:
df = df.filter(~col('VOTER_NAME').contains('_'))

In [23]:
df.select('VOTER_NAME').distinct().show(10, truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|VOTER_NAME                                                                                                                                                                                                                                                                                                                                                                                                   |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

8. Tạo cột splits chứa thông tin được cắt theo khoảng trắng VOTER_NAME

In [24]:
df = df.withColumn('splits', split(df.VOTER_NAME, '\s+'))

9. tạo cột 'first_name' lấy dữ liệu phần tử đầu tiên trong cột splits

In [25]:
df = df.withColumn('first_name', df.splits.getItem(0))

10. tạo cột 'last_name' lấy dữ liệu phần tử cuối trong cột splits

In [28]:
df = df.withColumn('last_name', df.splits.getItem(size('splits') - 1))

In [29]:
df.show(5)

+----------+--------------------+------------------+--------------------+----------+---------+
|      DATE|               TITLE|        VOTER_NAME|              splits|first_name|last_name|
+----------+--------------------+------------------+--------------------+----------+---------+
|04/11/2018|Deputy Mayor Pro Tem|      Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|
|02/14/2018|       Councilmember|   Lee M. Kleinman| [Lee, M., Kleinman]|       Lee| Kleinman|
|04/25/2018|       Councilmember|    Tennell Atkins|   [Tennell, Atkins]|   Tennell|   Atkins|
|08/29/2018|       Councilmember|      Kevin Felder|     [Kevin, Felder]|     Kevin|   Felder|
|10/18/2017|       Councilmember|Jennifer S.  Gates|[Jennifer, S., Ga...|  Jennifer|    Gates|
+----------+--------------------+------------------+--------------------+----------+---------+
only showing top 5 rows



11. Tạo col random_val theo điều kiện nếu TITLE == 'Councilmember' thì random_val có giá trị rand(), nếu TITLE == 'Mayor' thì random_val có giá trị 2, còn lại là 0

In [30]:
df = df.withColumn('random_val', when(df.TITLE == 'Councilmember', rand()))

In [31]:
df.show(5)

+----------+--------------------+------------------+--------------------+----------+---------+------------------+
|      DATE|               TITLE|        VOTER_NAME|              splits|first_name|last_name|        random_val|
+----------+--------------------+------------------+--------------------+----------+---------+------------------+
|04/11/2018|Deputy Mayor Pro Tem|      Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|              null|
|02/14/2018|       Councilmember|   Lee M. Kleinman| [Lee, M., Kleinman]|       Lee| Kleinman|   0.9013471272704|
|04/25/2018|       Councilmember|    Tennell Atkins|   [Tennell, Atkins]|   Tennell|   Atkins|0.7314561672035598|
|08/29/2018|       Councilmember|      Kevin Felder|     [Kevin, Felder]|     Kevin|   Felder|0.3081695631632436|
|10/18/2017|       Councilmember|Jennifer S.  Gates|[Jennifer, S., Ga...|  Jennifer|    Gates| 0.860288910605178|
+----------+--------------------+------------------+--------------------+----------+----

In [32]:
df = df.withColumn('random_val', when(df.TITLE == 'Councilmember', rand()).when(df.TITLE == 'Mayor', 2).otherwise(0))

In [33]:
df.show(5)

+----------+--------------------+------------------+--------------------+----------+---------+-------------------+
|      DATE|               TITLE|        VOTER_NAME|              splits|first_name|last_name|         random_val|
+----------+--------------------+------------------+--------------------+----------+---------+-------------------+
|04/11/2018|Deputy Mayor Pro Tem|      Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|                0.0|
|02/14/2018|       Councilmember|   Lee M. Kleinman| [Lee, M., Kleinman]|       Lee| Kleinman| 0.5633590066943959|
|04/25/2018|       Councilmember|    Tennell Atkins|   [Tennell, Atkins]|   Tennell|   Atkins|0.15074756624353658|
|08/29/2018|       Councilmember|      Kevin Felder|     [Kevin, Felder]|     Kevin|   Felder|0.12941809055441789|
|10/18/2017|       Councilmember|Jennifer S.  Gates|[Jennifer, S., Ga...|  Jennifer|    Gates|0.43906384201343185|
+----------+--------------------+------------------+--------------------+-------

12. Lọc random_val ==0, hiển thị

In [34]:
df.filter(df.random_val ==0).show(5)

+--------------------+--------------------+--------------------+--------------------+----------+---------+----------+
|                DATE|               TITLE|          VOTER_NAME|              splits|first_name|last_name|random_val|
+--------------------+--------------------+--------------------+--------------------+----------+---------+----------+
|          04/11/2018|Deputy Mayor Pro Tem|        Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|       0.0|
|A public hearing ...| authorize an  or...|  the  final   20...|[, the, final, 20...|          | District|       0.0|
|          04/12/2017|       Mayor Pro Tem|    Monica R. Alonzo|[Monica, R., Alonzo]|    Monica|   Alonzo|       0.0|
|          06/28/2017|Deputy Mayor Pro Tem|        Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|       0.0|
|          01/03/2018|Deputy Mayor Pro Tem|        Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|       0.0|
+--------------------+--------------------+-------------

13. Build function getFirstAndMiddle(names), trả về kqua First và Middle(names). Khai báo function dưới dạng udf, đặt tên là udfFirstAndMiddle

In [35]:
from pyspark.sql.types import *

In [36]:
def getFirstAndMiddle(names):
    return ' '.join(names[:-1])

In [37]:
udfFirstAndMiddle = udf(getFirstAndMiddle, StringType())

14. Tạo col first_and_middle_name bằng cách gọi udfFirstAndMiddle với tham số là cột splits. In kqua 

In [38]:
df = df.withColumn('first_and_middle_name', udfFirstAndMiddle(df.splits))

15. Xoá col first_name, splits. In kqua

In [39]:
df = df.drop('first_name')
df = df.drop('splits')


In [40]:
df.show(5)

+----------+--------------------+------------------+---------+-------------------+---------------------+
|      DATE|               TITLE|        VOTER_NAME|last_name|         random_val|first_and_middle_name|
+----------+--------------------+------------------+---------+-------------------+---------------------+
|04/11/2018|Deputy Mayor Pro Tem|      Adam Medrano|  Medrano|                0.0|                 Adam|
|02/14/2018|       Councilmember|   Lee M. Kleinman| Kleinman| 0.5633590066943959|               Lee M.|
|04/25/2018|       Councilmember|    Tennell Atkins|   Atkins|0.15074756624353658|              Tennell|
|08/29/2018|       Councilmember|      Kevin Felder|   Felder|0.12941809055441789|                Kevin|
|10/18/2017|       Councilmember|Jennifer S.  Gates|    Gates|0.43906384201343185|          Jennifer S.|
+----------+--------------------+------------------+---------+-------------------+---------------------+
only showing top 5 rows



In [41]:
df.select('VOTER_NAME').distinct()

DataFrame[VOTER_NAME: string]

In [42]:
print(df.count())

1272


16. thêm ROW_ID bằng phương thức monotonically_increasing_id

In [43]:
df = df.withColumn('ROW_ID', monotonically_increasing_id())

17. Hiển thị 10 rows đầu với ROW_ID tăng dần

In [44]:
df.orderBy(df.ROW_ID.desc()).show(10)

+----------+-------------+-------------------+---------+-------------------+---------------------+-------------+
|      DATE|        TITLE|         VOTER_NAME|last_name|         random_val|first_and_middle_name|       ROW_ID|
+----------+-------------+-------------------+---------+-------------------+---------------------+-------------+
|11/22/2016|Councilmember|       Mark Clayton|  Clayton| 0.9518419362182959|                 Mark|1709396983816|
|05/17/2017|Councilmember|       Lee Kleinman| Kleinman| 0.2065381667213041|                  Lee|1709396983815|
|03/22/2017|Councilmember| Rickey D. Callahan| Callahan| 0.6591820498571214|            Rickey D.|1709396983814|
|01/25/2017|Councilmember|      Sandy Greyson|  Greyson|0.08093150872391741|                Sandy|1709396983813|
|01/24/2018|Mayor Pro Tem|  Dwaine R. Caraway|  Caraway|                0.0|            Dwaine R.|1709396983812|
|12/13/2017|Councilmember|     Tennell Atkins|   Atkins| 0.0328348175000186|              Tennel

In [51]:
print('The number of Partitions in Data Frame: %d' % df.rdd.getNumPartitions())

The number of Partitions in Data Frame: 200


In [52]:
previous_max_ID = df.select('ROW_ID').rdd.max()[0]

In [53]:
voter_df_april = df.withColumn('ROW_ID',monotonically_increasing_id() + previous_max_ID )

In [55]:
df.select('ROW_ID').show(5)

+----------+
|    ROW_ID|
+----------+
|         0|
|         1|
|8589934592|
|8589934593|
|8589934594|
+----------+
only showing top 5 rows



In [56]:
voter_df_april.select('ROW_ID').show(5)

+-------------+
|       ROW_ID|
+-------------+
|1709396983816|
|1709396983817|
|1717986918408|
|1717986918409|
|1717986918410|
+-------------+
only showing top 5 rows

